In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
#Trend and Seasonality

def trend(time,slope=0):
    return slope * time

In [ ]:
def plot_series(time,series,format="-",start=0,end=None,label=None):
    plt.plot(time[start:end],series[start:end],format,label=label)
    plt.xlabel('Time')
    plt.ylabel('Value')
    if label:
        plt.legend(fontsize=14)
    plt.grid(True)    

In [ ]:
time = np.arange(4*365 + 1)
baseline = 10
series = trend(time,0.1)

plt.figure(figsize=(10,6))
plot_series(time,series)
plt.show

In [ ]:
#Time Series with seasonal pattern
def seasonal_pattern(season_time):
    return np.where(season_time < 0.4,
                   np.cos(season_time * 2 * np.pi),
                   1 / np.exp(3 * season_time))

def seasonality(time,period,amplitude=1,phase=0):
    season_time = ((time+phase) % period) / period
    return amplitude * seasonal_pattern(season_time)
    

In [ ]:
baseline = 10
amplitude = 40
series = seasonality(time,period=365,amplitude=amplitude)

plt.figure(figsize=(10,6))
plot_series(time,series)
plt.show()

In [ ]:
#Time series with both trend and seasonality

slope = 0.05
series = baseline + trend(time,slope) + seasonality(time,period=365,amplitude = amplitude)

plt.figure(figsize=(10,6))
plot_series(time,series)
plt.show()

In [ ]:
#Noise
def white_noise(time,noise_level=1,seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

In [ ]:
noise_level = 5
noise = white_noise(time,noise_level,seed=42)

plt.figure(figsize=(10,6))
plot_series(time,noise)
plt.show()

In [ ]:
#Let's add noise to the time series

series += noise

plt.figure(figsize=(10,6))
plot_series(time,series)
plt.show()

In [ ]:
#Split into 2 periods

split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

In [ ]:
def autocorrelation(time,amplitude,seed=None):
    rnd=np.random.RandomState(seed)
    s1  = 0.5
    s2  = -0.1
    ar  = rnd.randn(len(time)+50)
    ar[:50] = 100
    for step in range(50,len(time)+50):
        ar[step] += s1 * ar[step - 50]
        ar[step] += s2 * ar[step - 33]
        
    return ar[50:] * amplitude

In [ ]:
def autocorrelation(time,amplitude,seed=None):
    rnd = np.random.RandomState(seed)
    s = 0.8
    ar = rnd.randn(len(time) + 1)
    for step in range(1,len(time)+1):
        ar[step] += s * ar[step -1]
    return ar[1:] * amplitude    

In [ ]:
series = autocorrelation(time,10,seed=13)
plot_series(time[:200],series[:200])
plt.show()

In [ ]:
series = autocorrelation(time,10,seed=13) + trend(time,2)
plot_series(time[:200],series[:200])
plt.show()

In [ ]:
series = autocorrelation(time,10,seed=13) + seasonality(time,period=50,amplitude=150) + trend(time,2)
plot_series(time[:200],series[:200])
plt.show()

In [ ]:
series = autocorrelation(time,10,seed=13) + seasonality(time,period=50,amplitude=150) + trend(time,2)
series2 = autocorrelation(time,5,seed=13) + seasonality(time,period=50,amplitude=2) + trend(time,-1) + 550
series[200:] = series2[200:]
#series += noise(time,30)
plot_series(time[:300],series[:300])
plt.show()

In [ ]:
def impulses(time,num_impulses,amplitude=1,seed=None):
    rnd = np.random.RandomState(seed)
    impulse_indices = rnd.randint(len(time),size=10)
    series = np.zeros(len(time))
    for index in impulse_indices:
        series[index] += rnd.rand() * amplitude
    return series    

In [ ]:
series = impulses(time,10,seed=13)
plot_series(time,series)
plt.show()

In [ ]:
def autocorrelation(source,ss):
    ar = source.copy()
    max_leg= len(ss)
    for step, value in enumerate(source):
        for lag,s in ss.items():
            if step - lag >0:
                ar[step] += s * ar[step - lag]
    return ar            

In [ ]:
signal = impulses(time,10,seed=13)
series = autocorrelation(signal,{1:0.99})
plot_series(time,series)
plt.plot(time,signal,'k-')
plt.show()


In [ ]:
series_diff1 = series[1:] - series[:-1]
plot_series(time[1:],series_diff1)

In [ ]:
from pandas.plotting import autocorrelation_plot

autocorrelation_plot(series)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

model= ARIMA(series,order = (5,1,0))
model_fit=model.fit(disp=0)
print(model_fit.summary())

In [ ]:
df = pd.read_csv('/kaggle/input/sunspots/Sunspots.csv',parse_dates=['Date'],index_col='Date')
series = df['Monthly Mean Total Sunspot Number'].asfreq('1M')
series.head()

In [ ]:
series.plot(figsize=(12,5))

In [ ]:
series['1995-01-01':].plot()

In [ ]:
series.diff(1).plot()
plt.axis([0,100,-50,50])

In [ ]:
from pandas.plotting import autocorrelation_plot

autocorrelation_plot(series)

In [ ]:
autocorrelation_plot(series.diff(1)[1:])

In [ ]:
autocorrelation_plot(series.diff(1)[1:].diff(11*12)[11*12+1:])
plt.axis([0,500,-0.1,0.1])

In [ ]:
autocorrelation_plot(series.diff(1)[1:])
plt.axis([0,50,-0.1,0.1])